# 프로젝트 3. 자연어 분류 Sentiment Analysis on Movie Reviews (kaggle)

## 코딩은 처음이라 with 딥러닝 (이종환 저)

http://www.yes24.com/Product/Goods/108837747

<img src="https://i.imgur.com/6hWiXwC.png" width="600px">

## Sentiment Analysis on Movie Reviews

https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews

1. [경진대회](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews) 참가
2. 우측 Setting 메뉴 - Internet on (V)
3. [선택] 우측 Setting 메뉴 Accelartor - GPU

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 1. Data Load 및 확인

In [ ]:
train = pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip')
test = pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip')
sub = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv')

train.head()

In [ ]:
train['Sentiment'].value_counts()

In [ ]:
sub.head()

## SentenceBERT 를 사용한 분류

In [ ]:
!pip install -q sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

sentences = ["This is an example sentence", "Each sentence is converted"]

sbert = SentenceTransformer('sentence-transformers/sentence-t5-base')
embeddings = sbert.encode(sentences)

print(embeddings.shape)

In [ ]:
encoded_phrases = sbert.encode(train['Phrase'])

print(encoded_phrases.shape)

In [ ]:
encoded_phrases_test = sbert.encode(test['Phrase'])

print(encoded_phrases_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(encoded_phrases, train['Sentiment'], test_size=0.2, random_state=42)

print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import *

model3 = Sequential([
    Dense(512, input_shape=(768,), activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax'),
])

model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='acc')

model3.summary()

In [ ]:
history3 = model3.fit(train_x, train_y, epochs=30, batch_size=1024, validation_data=(val_x, val_y))
result3 = model3.predict(encoded_phrases_test)

In [ ]:
sub['Sentiment'] = result3.argmax(1)
sub.to_csv('sent3.csv', index=False)